# IMPORTATION DES DONNEES ET MERGE

In [1]:
#Import de toutes les librairies nécessaires

import pandas as pd
import numpy as np
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.feature_selection import SelectKBest, f_classif, SelectFromModel
from sklearn.linear_model import SGDClassifier 
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

from collections import Counter
from xgboost import plot_importance
from matplotlib import pyplot


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#Création des DF
df_users=pd.read_csv("users.csv")
df_places=pd.read_csv("places.csv",low_memory=False)
df_caract=pd.read_csv("caracteristics.csv",encoding='ISO-8859-1',low_memory=False)
df_veh=pd.read_csv("vehicles.csv")

In [3]:
#Merge des données

df_projet_brut=pd.merge(df_users, df_veh, how='left', on=["Num_Acc", "num_veh"])
df_projet_brut=pd.merge(df_projet_brut, df_caract, how='left', on=["Num_Acc"])
df_projet_brut=pd.merge(df_projet_brut, df_places, how='left', on=["Num_Acc"])

# PREPOCESSING & FEATURES SELECTION

In [4]:
#On nettoie le fichier df_projet_brut
    #On supprime les variables gps, lat et long car il y a beaucoup de NaN et pas utile pour la gravité car variable de géolocalisation
    #On supprime les variables voie, v1 et v2 car il y a beaucoup de NaN et pas utiles pour la gravité car des variables indiquant des numéros de rue comme la variable adr_car
    #On supprime les variables pr et pr1 car beaucoup de NaN et pas utlies pour la gravité car des variables relative au lieu d'habitation.
    #On supprime la variable infra car beaucoup de valeurs en 0 (valeurs non indiquées dans les définitions.
    #On supprime la variable Num_Acc car ce n'est pas une variable explicative
    #On supprume la variable com (Comummune) fortement correlée avec dep (departement) 
    #On supprume la variable nbv car beaucoup de catégories (53) et les informations que nous avons besons sont dans circ(type de route avec nb de voie)
    #On supprime la variable senc (sens de circulation) énormement d'inconnu a + de 90%
    #On suppriume la varibale place (place dans le vehicule) car la variable catu donne deja le type de user a savoir, conducteur, passager

to_drop=['gps','lat','long','voie','v1','v2','adr','pr','pr1','infra','Num_Acc','com', 'nbv','senc','place']
df_projet_brut_2=df_projet_brut.drop(to_drop,axis=1)

In [5]:
# Retraitement Variables:

# catv (99 catégories) on retraire en: vehicule 2 roues, vehicule non carrossé (hors 2 roues), 
# vehicule leger carrosé, vehicule leger carrosé avec attelage (remorque, caravanne), poids lourd et transport en commun.
# 2 Roues,1,Commun,3,Inconnu,0,Poids Lourds,4,VL Attelage,5,VL Carrose,2,VL Non Carrose,6,

#catv
catv_value = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,30,31,32,33,34,35,36,37,38,39,40,41,42,43,50,60,80,99]
catv_value_new = [0,1,1,2,1,1,6,2,5,5,5,5,5,4,4,4,4,4,3,3,4,4,1,1,1,1,1,6,6,3,3,3,3,1,1,1,1,1,0,0]
df_projet_brut_2['catv'].replace(catv_value, catv_value_new, inplace = True)


# plan = type de voie (rectiligne ou type de courbes), je propose de faire 0 rectiligne 1 courbe

#plan
plan_value = [1,2,3,4]
plan_value_new = [0,1,1,1]
df_projet_brut_2['plan'].replace(plan_value, plan_value_new, inplace = True)

# Retraitement secu. Le code secu appliqué dans les données de Kaggle a peu de sens
# On crée danc une nlle va secu indiquant si existance d'un materiel de securité et utilisé (casque, ceinture de secu, etc)

# Valeurs actuelles
secu_value = df_projet_brut_2['secu'].value_counts().index.tolist()

# Nlle valeur indique si utilisation d'un element de scurité (0) sinon (1) (1 integre la non connaissance et nan)
secu_new = [0, 0, 1,1,1,1,1,1,0,0,1,1,0,0,1,1,1,1,1,1,1,1,1,1]

#Nlle colonne new_secu

df_projet_brut_2['New_secu'] = df_projet_brut_2['secu']

df_projet_brut_2['New_secu'].replace(secu_value, secu_new, inplace = True)
df_projet_brut_2 = df_projet_brut_2.drop('secu', axis = 1)

#Grav
df_projet_brut_2['grav'].replace([1, 2, 3,4],[0, 1, 1, 0], inplace=True)


In [12]:
# Travail sur les valeurs de la variable num_veh

num_veh = df_projet_brut['num_veh'].value_counts(normalize = True)*100 #en dessous de la 5eme valeur on descend à une contribution de moins de 1%

X_num=df_projet_brut_2.select_dtypes(include=['float64','int64'])
X_cat=df_projet_brut_2.select_dtypes('object')
X_cat=pd.get_dummies(X_cat)
df_projet_brut_3=pd.concat([X_num,X_cat], axis=1)

num_veh_to_drop = num_veh.tail(124).index
append_str = 'num_veh_'
num_veh_to_drop = [append_str + sub for sub in num_veh_to_drop]
df_projet_brut_3 = df_projet_brut_3.drop(num_veh_to_drop, axis = 1)

df_projet_brut_3 = df_projet_brut_3.dropna(axis = 1, how='all')
df_projet_brut_3 = df_projet_brut_3.dropna(axis = 0, how='any')

target=df_projet_brut_3['grav']
feats=df_projet_brut_3.drop(['grav'],1)


/var/folders/65/xmhb__dx72g7hbmbpm_rkmgw0000gn/T/ipykernel_2494/1769733279.py:19: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  feats=df_projet_brut_3.drop(['grav'],1)


In [ ]:
select = SelectKBest(f_classif, k= 20)

# ici l'idée n'est pas de faire un train et test mais de capturer les features les plus contingentes à la target
# Donc on prends le dataset complet avec la target dualisée (Grave vs Pas grave)

select.fit_transform(feats, target)

feats_KBest = feats.columns[select.get_support()]
print("les 20 va les plus explicatives selectionnées sont:", feats.columns[select.get_support()])

les 20 va les plus explicatives selectionnées sont: Index(['catu', 'locp', 'actp', 'etatp', 'catv', 'obs', 'obsm', 'agg', 'int',
       'atm', 'col', 'dep', 'catr', 'plan', 'situ', 'New_secu', 'num_veh_A01',
       'num_veh_B01', 'num_veh_B02', 'num_veh_C01'],
      dtype='object')


In [ ]:
# Selection des best features selon SelectFromModel

selector_mean = SelectFromModel(SGDClassifier(random_state = 0), threshold = 'mean')

selector_mean.fit_transform(feats, target)

feats_sgdc_mean = feats.columns[selector_mean.get_support()]
print("les va les plus explicatives selectionnées sont:", feats.columns[selector_mean.get_support()])

/Users/fabricebazinimac/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


les va les plus explicatives selectionnées sont: Index(['catu', 'sexe', 'etatp', 'catv', 'agg', 'plan', 'situ', 'New_secu',
       'num_veh_A01', 'num_veh_B02', 'num_veh_C01'],
      dtype='object')


In [ ]:
# Selection des best features par la mediane pour des va catégorielles

selector_median = SelectFromModel(SGDClassifier(random_state = 0), threshold = 'median')

selector_median.fit_transform(feats, target)

feats_sgdc_med = feats.columns[selector_median.get_support()]
print("les va les plus explicatives selectionnées par mediane sont:", feats.columns[selector_median.get_support()])



/Users/fabricebazinimac/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


les va les plus explicatives selectionnées par mediane sont: Index(['catu', 'sexe', 'locp', 'actp', 'etatp', 'catv', 'obs', 'obsm', 'agg',
       'col', 'circ', 'vosp', 'plan', 'situ', 'New_secu', 'num_veh_A01',
       'num_veh_B01', 'num_veh_B02', 'num_veh_C01', 'num_veh_C03'],
      dtype='object')


In [ ]:
# Utilisation d'un modele de Gradient Boosting (particulièrement adapté aux va cat par tree decision) avec New_secu

#n_estimators=20, learning_rate=learning_rate, max_features=2, max_depth=2, random_state=0
gb_clf = SelectFromModel(GradientBoostingClassifier())
gb_clf.fit_transform(feats, target)

feats_gbc = feats.columns[gb_clf.get_support()]
print("les va les plus explicatives selectionnées par GBC sont:", feats.columns[gb_clf.get_support()])

les va les plus explicatives selectionnées par GBC sont: Index(['catu', 'catv', 'obs', 'agg', 'col', 'dep', 'catr', 'num_veh_B02'], dtype='object')


In [ ]:
# Feature Selection : On crée un DF ne contenant que les features selectionnées precedemment

keep_feats0 = feats_KBest.append(feats_sgdc_mean)
keep_feats1 = keep_feats0.append(feats_sgdc_med)
keep_feats = keep_feats1.append(feats_gbc)

keep_feats = Counter(keep_feats).keys()

print(keep_feats)

feats_select = feats[keep_feats]

dict_keys(['catu', 'locp', 'actp', 'etatp', 'catv', 'obs', 'obsm', 'agg', 'int', 'atm', 'col', 'dep', 'catr', 'plan', 'situ', 'New_secu', 'num_veh_A01', 'num_veh_B01', 'num_veh_B02', 'num_veh_C01', 'sexe', 'circ', 'vosp', 'num_veh_C03'])


In [14]:
# On garde les features sans les nim_veh

keep=['catu', 'locp', 'actp', 'etatp', 'catv', 'obs', 'obsm', 'agg', 'int', 'atm', 'col', 'dep', 'catr', 'plan', 'situ', 'New_secu', 'sexe', 'circ', 'vosp']

feats_select = feats[keep]

feats_select.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1807155 entries, 0 to 1876004
Data columns (total 19 columns):
 #   Column    Dtype  
---  ------    -----  
 0   catu      int64  
 1   locp      float64
 2   actp      float64
 3   etatp     float64
 4   catv      float64
 5   obs       float64
 6   obsm      float64
 7   agg       int64  
 8   int       int64  
 9   atm       float64
 10  col       float64
 11  dep       int64  
 12  catr      float64
 13  plan      float64
 14  situ      float64
 15  New_secu  float64
 16  sexe      int64  
 17  circ      float64
 18  vosp      float64
dtypes: float64(14), int64(5)
memory usage: 275.7 MB


In [15]:
# Création des jeux d'entrainement et de test

X_train_select, X_test_select, y_train_select, y_test_select = train_test_split(feats_select, target, test_size=0.20, random_state=1234)



# XGBoost

In [ ]:
#Ce code a mis 161 min à calculer

#Détermination des meilleurs paramètres pour XGboost

# PARAMETERS = {"subsample":[0.5, 0.75, 1],"colsample_bytree":[0.5, 0.75, 1],
#             "max_depth":[2, 6, 12], "learning_rate":[0.3, 0.1, 0.03], "n_estimators":[100]}


# model = xgb.XGBClassifier(n_estimators=100, n_jobs=-1)
# model_gs = GridSearchCV(model,param_grid=PARAMETERS,cv=3,scoring="accuracy")
# model_gs.fit(X_train,y_train)

# print(model_gs.best_params_)

In [16]:
#Entrainement et prédiction avec les paramètres optimaux déterminés précédemment

xg_reg = xgb.XGBClassifier(colsample_bytree= 0.5,
                        learning_rate= 0.1,
                        max_depth= 12,
                        n_estimators= 100,
                        subsample= 1)

In [17]:
# Entrainement du modèle et prédiction
xg_reg.fit(X_train_select,y_train_select)

pred_values = xg_reg.predict(X_test_select)

In [18]:
#Affichage du Classification Report

print(metrics.classification_report(y_test_select, pred_values))

              precision    recall  f1-score   support

           0       0.85      0.94      0.89    277548
           1       0.68      0.44      0.53     83883

    accuracy                           0.82    361431
   macro avg       0.76      0.69      0.71    361431
weighted avg       0.81      0.82      0.81    361431



In [19]:
#Affichage de la Crosstab
pd.crosstab(y_test_select, pred_values, rownames=['CLasse réelle'], colnames=['Classe prédite'])

Classe prédite,0,1
CLasse réelle,,
0,260028,17520
1,47230,36653
